In [18]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import optuna
from optuna.samplers import GridSampler

In [3]:
data = pd.read_csv(r"C:\Users\Varshitha\Desktop\gifb ml\hand_detection.csv")

In [5]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,label
0,0.770949,0.792555,-1.317012e-06,0.649365,0.701700,-0.028799,0.561749,0.588996,-0.058532,0.525716,...,0.674526,0.461092,-0.119693,0.682588,0.534539,-0.107359,0.723431,0.544322,-0.087382,rock
1,0.780138,0.793937,-9.804838e-07,0.658679,0.698270,-0.018191,0.573301,0.588336,-0.042320,0.524413,...,0.647207,0.472924,-0.096524,0.660742,0.544293,-0.088038,0.706911,0.552741,-0.072803,rock
2,0.828314,0.831293,-1.140637e-06,0.701673,0.717367,0.013490,0.634446,0.604296,-0.003696,0.581749,...,0.618503,0.530582,-0.116007,0.636557,0.604018,-0.103250,0.686282,0.614387,-0.088857,rock
3,0.839761,0.832153,-1.037426e-06,0.724383,0.717950,0.012774,0.656546,0.606731,-0.007092,0.599371,...,0.622043,0.536777,-0.107738,0.644623,0.613937,-0.096987,0.693016,0.622973,-0.085781,rock
4,0.872746,0.809625,-7.791157e-07,0.839749,0.662170,0.014471,0.765383,0.528655,-0.004376,0.660104,...,0.633462,0.536624,-0.102321,0.656336,0.603833,-0.094508,0.700882,0.613244,-0.089441,rock


In [6]:
x = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [7]:
x

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,0.770949,0.792555,-1.317012e-06,0.649365,0.701700,-0.028799,0.561749,0.588996,-0.058532,0.525716,...,-0.082254,0.674526,0.461092,-0.119693,0.682588,0.534539,-0.107359,0.723431,0.544322,-0.087382
1,0.780138,0.793937,-9.804838e-07,0.658679,0.698270,-0.018191,0.573301,0.588336,-0.042320,0.524413,...,-0.064982,0.647207,0.472924,-0.096524,0.660742,0.544293,-0.088038,0.706911,0.552741,-0.072803
2,0.828314,0.831293,-1.140637e-06,0.701673,0.717367,0.013490,0.634446,0.604296,-0.003696,0.581749,...,-0.091449,0.618503,0.530582,-0.116007,0.636557,0.604018,-0.103250,0.686282,0.614387,-0.088857
3,0.839761,0.832153,-1.037426e-06,0.724383,0.717950,0.012774,0.656546,0.606731,-0.007092,0.599371,...,-0.088740,0.622043,0.536777,-0.107738,0.644623,0.613937,-0.096987,0.693016,0.622973,-0.085781
4,0.872746,0.809625,-7.791157e-07,0.839749,0.662170,0.014471,0.765383,0.528655,-0.004376,0.660104,...,-0.088505,0.633462,0.536624,-0.102321,0.656336,0.603833,-0.094508,0.700882,0.613244,-0.089441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.733321,0.738820,2.211360e-07,0.679061,0.614360,0.040242,0.591085,0.580070,0.059709,0.525282,...,-0.007681,0.487520,0.749686,0.011763,0.514990,0.746789,0.026750,0.548304,0.747146,0.037398
596,0.742377,0.760697,1.440724e-07,0.698196,0.645465,0.032491,0.622427,0.603259,0.050446,0.558952,...,-0.000128,0.506152,0.768484,0.020253,0.531657,0.776545,0.035486,0.562575,0.781978,0.047007
597,0.744655,0.763418,1.372435e-07,0.699988,0.653593,0.038801,0.625520,0.619293,0.061509,0.569003,...,0.011503,0.524989,0.769254,0.032156,0.548732,0.775755,0.044558,0.577733,0.780757,0.053761
598,0.741164,0.764988,1.938232e-07,0.699515,0.661214,0.033506,0.625542,0.630267,0.051828,0.566710,...,0.005989,0.529613,0.772600,0.024590,0.553939,0.777331,0.035636,0.582406,0.778951,0.043632


In [8]:
y

0         rock
1         rock
2         rock
3         rock
4         rock
        ...   
595    scissor
596    scissor
597    scissor
598    scissor
599    scissor
Name: label, Length: 600, dtype: object

In [11]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [12]:
space = {
    'n_estimators': [60, 120, 180],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [13]:
sampler = GridSampler(space)


In [27]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', space['n_estimators']),
        'max_depth': trial.suggest_categorical('max_depth', space['max_depth']),
        'min_samples_split': trial.suggest_categorical('min_samples_split',space['min_samples_split']),
        'min_samples_leaf': trial.suggest_categorical('min_samples_leaf', space['min_samples_leaf'])
    }


    model = RandomForestClassifier(**params, random_state=42)
    scores = cross_val_score(model, x, y_encoded, cv=3, scoring='accuracy')
    return scores.mean()

In [28]:
study = optuna.create_study(direction="maximize",sampler=sampler)
study.optimize(objective,n_trials=len(sampler._all_grids))

[I 2025-07-17 15:17:58,272] A new study created in memory with name: no-name-f9600afe-82ed-4573-a518-fa784f614790
[I 2025-07-17 15:17:58,951] Trial 0 finished with value: 0.9366666666666666 and parameters: {'n_estimators': 120, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.9366666666666666.
[I 2025-07-17 15:17:59,855] Trial 1 finished with value: 0.93 and parameters: {'n_estimators': 180, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.9366666666666666.
[I 2025-07-17 15:18:00,625] Trial 2 finished with value: 0.935 and parameters: {'n_estimators': 120, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.9366666666666666.
[I 2025-07-17 15:18:00,950] Trial 3 finished with value: 0.9333333333333335 and parameters: {'n_estimators': 60, 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.9366666666666666.
[I 202

In [29]:
best_params=study.best_params
print("best hyperparameter =",best_params)

best hyperparameter = {'n_estimators': 60, 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 1}


In [32]:
final_model = RandomForestClassifier(**best_params,random_state=42)
final_model.fit(x,y_encoded)
y_predict = final_model.predict(x)
acc= accuracy_score(y_encoded,y_predict)
print("training accuracy",acc)

training accuracy 1.0


In [33]:
with open("hand_sign_model.pkl", "wb") as h:
    pickle.dump(final_model, h)

with open("label_encoder.pkl", "wb") as h:
    pickle.dump(label_encoder, h)

print("Model and label encoder saved.")


Model and label encoder saved.


In [38]:
import cv2
import numpy as np
import pandas as pd
import pickle
import mediapipe as mp

with open("hand_sign_model.pkl", "rb") as h:
    model = pickle.load(h)

with open("label_encoder.pkl", "rb") as h:
    label_encoder = pickle.load(h)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


# MediaPipe Hands model to detect hands in a video 

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)

cap = cv2.VideoCapture(1)

# Checks if any hand landmarks were detected (multi_hand_landmarks).

# If yes, it takes only the first detected hand.

# Loops through each of the 21 landmarks in that hand.

# For each landmark, it appends the x, y, and z coordinates to a list.

# Converts the list to a 1-row NumPy array with shape (1, 63) — since 21 landmarks × 3 values = 63.

def extract_landmarks(results):
    if not results.multi_hand_landmarks:
        return None
    hand_landmarks = results.multi_hand_landmarks[0]
    landmarks = []
    for lm in hand_landmarks.landmark:
        landmarks.extend([lm.x, lm.y, lm.z])
    return np.array(landmarks).reshape(1, -1)
    
    
while True:
    ret, frame = cap.read()
    if not ret:  #If a frame is not read successfully
        break
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    result = hands.process(frame_rgb)

    if result.multi_hand_landmarks:  #Checks if MediaPipe successfully detected any hands in the current frame.If no hands, it skips drawing.
        
        for hand_landmarks in result.multi_hand_landmarks:  #Loops over all detected hands ,each hand_landmarks contains 21 points for a hand 
            
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)  # It draws 21 landmarks (dots at joints)Connections between landmarks
            
            features = extract_landmarks(result) #extract hand landmarks and output will be array
            
            if features is not None:  # continue only if hand is detected
                
                features_df = pd.DataFrame(features, columns=[str(i) for i in range(features.shape[1])]) # creates col names with 1-62
                
                pred = model.predict(features_df)  #uses model to predict the gestures
                
                label = label_encoder.inverse_transform(pred)[0] # convets the numerical labels to original names 
                
                cv2.putText(frame, f"Prediction: {label}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
                cv2.imshow("Rock Paper Scissors Detection", frame)

    if cv2.waitKey(1) & 255 == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

    



In [ ]:
import cv2

# Open the default camera (0 is usually the built-in webcam)
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open video capture")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # If frame is read correctly, ret is True
    if not ret:
        print("Error: Can't receive frame (stream end?). Exiting ...")
        break

    # Display the resulting frame
    cv2.imshow('Webcam Feed', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture object and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()
